To Do:  
* move video rotation.
* Legend.
* Check dates competibilities color to mesh.
* calc clim

test the basic functions needed to create the cube rendering

In [100]:
import pyvista as pv
import numpy as np
import netCDF4 as nc
import vtk
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from skimage import io
import cv2
import logging
import pandas as pd
from scipy.interpolate import griddata
from scipy.spatial import QhullError
#import gemgis as gg

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Parameters

In [2]:
bg_padding = 400
cube_size = 10
cmap = 'seismic' #'RdBu'# 'PiYG' 'seismic_r'
clim = (-100,100)
clim = (0,2.6)
crop_coordinates = None # (40,55,0,20)

n_timepoints=100
frame_rate=30
event_csv_path='events.csv'

path_input_mesh = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
#path_input_color = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
path_input_color = '../../UFZ_RemoteSensing/NOAA-LAI-Europe-mon-2001-2012(1).nc'

output_path="output_cube10_seismic_bgPad400.mp4"



In [48]:
bg_padding = 220
cube_size = 20
cmap = 'PiYG' #'RdBu'# 'PiYG' 'seismic_r'
clim = (-100,100)
clim = (0,2.6)
crop_coordinates = (40,59,0,21)

n_timepoints=20
frame_rate=30
event_csv_path=None#'events.csv'

is_interp = 'spatial'

path_input_mesh = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
#path_input_color = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
path_input_color = '../../UFZ_RemoteSensing/NOAA-LAI-Europe-mon-2001-2012(1).nc'

output_path="output_cube20_PiYG_bgPad200_cropped.mp4"

In [4]:
path_input_BG = '../backgound_map_tests/bluemarble.png'

## Functions

### Load, crop & resize data

In [5]:
def load_nc(path_input):
    ds = nc.Dataset(path_input)
    arr_name = [n for n in ds.variables.keys() if ds[n].ndim==3][0]
    arr = ds[arr_name]

    lat_arr = ds['latitude'][:]
    long_arr = ds['longitude'][:]
    time_arr = [s[:-2] for s in ds['time'][:].astype(str)]
    
    logging.info(f'Read netCDF {arr_name} data. Shape={ds[arr_name].shape}')
    
    return arr, lat_arr, long_arr, time_arr

In [70]:
def load_arr_2D(arr, arr_slice=0, fill_val=-9999, min_zero=False):
    arr = arr[:][arr_slice]
    arr = arr.filled(fill_value=np.nan)
    arr = arr - np.nanmin(arr) if min_zero else arr
    
    return arr

In [7]:
def find_min_max(coor_arr_mesh, coor_arr_color, crop_coordinates):
    coor_min = np.nanmax((np.nanmin(coor_arr_mesh), np.nanmin(coor_arr_color), crop_coordinates[0]))
    coor_max = np.nanmin((np.nanmax(coor_arr_mesh), np.nanmax(coor_arr_color), crop_coordinates[1]))
    
    return coor_min, coor_max

In [8]:
def find_idx_nearest(arr, val):
    return (np.abs(arr - val)).argmin()

def get_world_part(arr, lat_arr_src, long_arr_src, 
                   lat_arr_dst=None, long_arr_dst=None,
                   padding=0):
    
    if lat_arr_dst is None:
        lat_arr_dst = np.linspace(-90, 90, arr.shape[0])
        long_arr_dst = np.linspace(-180, 180, arr.shape[1])

    min_lat_idx = find_idx_nearest(lat_arr_dst, np.min(lat_arr_src))
    max_lat_idx = find_idx_nearest(lat_arr_dst, np.max(lat_arr_src))

    min_long_idx = find_idx_nearest(long_arr_dst, np.min(long_arr_src))
    max_long_idx = find_idx_nearest(long_arr_dst, np.max(long_arr_src))

    arr = arr[min_lat_idx-padding:max_lat_idx+padding, min_long_idx-padding:max_long_idx+padding]
    
    return arr

In [9]:
def make_BG_mesh(path_input_BG, bg_padding, lat_min, lat_max, long_min, long_max):
    bg = np.flip(io.imread(path_input_BG), axis=0)
    bg = get_world_part(bg, (lat_min, lat_max), (long_min, long_max), padding=bg_padding)
    data_shape = [s-bg_padding*2 for s in bg.shape[:2]]
    path_bg = 'tmp.png'
    io.imsave(path_bg, np.flip(bg,axis=0))
    bg = pv.read(path_bg)
    bg.origin = (-bg_padding, -bg_padding, 0)
    
    return bg, data_shape

In [109]:
def interpolate_nans(arr_color):
    
    if ~np.all(np.isnan(arr_color)):
        
        idxs_to_interp = np.where(np.isnan(arr_color))
        idxs_interp_from = np.where(~np.isnan(arr_color))
        
        try:
            arr_color[idxs_to_interp] = griddata(idxs_interp_from, 
                                             arr_color[idxs_interp_from], 
                                             idxs_to_interp)
        except QhullError:
            logging.warning('not interpulating - QHull error')
            
    return arr_color

In [110]:
def interpolate_nans_temporal(arr_color_ref):
    
    for idx in range(arr_color_ref.shape[-1]):
        arr_color = load_arr_2D(arr_color_ref, np.index_exp[:,:,idx])
        interpolate_nans(arr_color)
        

In [ ]:
arr_color_ref[:]

### Make plotter

In [10]:
def create_plotter(frame_rate):
    p = pv.Plotter(notebook=False, off_screen=True,
                  window_size=[1920 * 2, 1080 * 2], 
                   multi_samples=16, lighting='three lights')
    p.open_movie(output_path, framerate=frame_rate)
    p.set_background('black')
    
    return p

In [11]:
def add_BG(p, bg):
    floor_thickness = 40
    p.add_mesh(pv.Cube(center=(bg.center[0], bg.center[1], -floor_thickness), 
                       x_length=bg.bounds[1]-bg.bounds[0],
                       y_length=bg.bounds[3]-bg.bounds[2], 
                       z_length=floor_thickness*2-0.02),
               color='white')

    p.add_mesh(bg, rgb=True, scalars='PNGImage')
    
    return p

### Make mesh

In [12]:
def make_floor_vertices(arr, cube_size):

    xs,ys = np.meshgrid(range(arr.shape[0]),range(arr.shape[1]))
    
    vertices_base = np.vstack((xs.T.flatten()*cube_size, 
                        ys.T.flatten()* cube_size,
                        np.zeros(xs.size))).T
    
    xs,ys = np.meshgrid(range(arr.shape[0]),[arr.shape[1]])
    vertices_edge_y = np.vstack((xs.T.flatten()*cube_size, 
                        ys.T.flatten()*cube_size,
                        np.zeros(xs.size))).T
    
    xs,ys = np.meshgrid(arr.shape[0],range(arr.shape[1]))
    vertices_edge_x = np.vstack((xs.T.flatten()* cube_size, 
                        ys.T.flatten()* cube_size,
                        np.zeros(xs.size))).T
        
    return vertices_base, vertices_edge_y, vertices_edge_x

In [13]:
def concat_all_vertices(arr, vertices_base, vertices_edge_y, vertices_edge_x, cube_size=1):
    
    vertices_ceil = vertices_base.copy()
    vertices_ceil[:,2] = (arr.flatten()/10)**2-(np.nanmin(arr.flatten()/10)**2)
    
    vertices = np.concatenate((vertices_base,
                               vertices_edge_y,
                               vertices_edge_x,
                               vertices_ceil,
                               vertices_ceil+[0,1*cube_size,0],
                               vertices_ceil+[1*cube_size,1*cube_size,0],
                               vertices_ceil+[1*cube_size,0,0],
                          ))
    
    return vertices

In [14]:
def make_vertices(arr, cube_size=1):
    vertices_base, vertices_edge_y, vertices_edge_x = make_floor_vertices(arr, cube_size)
    vertices = concat_all_vertices(arr, vertices_base, vertices_edge_y, vertices_edge_x, cube_size)
    
    n_floor_v = vertices_base.shape[0]+vertices_edge_y.shape[0]+vertices_edge_x.shape[0]
    return vertices, n_floor_v

In [15]:
def make_floor_faces(arr):
    faces = []
    it = np.nditer(arr, flags=['c_index','multi_index'])

    for x in it:
        if not np.isnan(x):
            
            neigh_right_idx = it.index+1 if it.multi_index[1]!=arr.shape[1]-1 \
                                else arr.size + it.multi_index[0] 
            neigh_down_idx = it.index+arr.shape[1] if it.multi_index[0]!=arr.shape[0]-1 \
                                else arr.size + arr.shape[0] + it.multi_index[1]
            
            neigh_diag_idx = it.index+arr.shape[1]+1
            if neigh_down_idx>arr.size:
                neigh_diag_idx = neigh_down_idx+1
            elif neigh_right_idx>arr.size-1:
                neigh_diag_idx = neigh_right_idx+1

            faces.append(np.array([4,
                                   it.index,
                                   neigh_right_idx,
                                   neigh_diag_idx,
                                   neigh_down_idx,
                                  ]))
            
    return faces

In [16]:
def make_ceil_faces(arr, n_v_floor):

    faces = []
    it = np.nditer(arr, flags=['c_index','multi_index'])

    for x in it:
        if not np.isnan(x):
            faces.append(np.array([4,
                                   n_v_floor+it.index,
                                   n_v_floor+arr.size+it.index,
                                   n_v_floor+arr.size*2+it.index,
                                   n_v_floor+arr.size*3+it.index,
                                  ]))
    
    return faces

In [17]:
def make_side_faces(faces):
    side_faces = []

    for i in range(len(faces)//2):
        side_faces.append(np.array([4,
                               faces[i][1],
                               faces[i][2],
                               faces[len(faces)//2+i][2],
                               faces[len(faces)//2+i][1],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][1],
                               faces[i][4],
                               faces[len(faces)//2+i][4],
                               faces[len(faces)//2+i][1],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][2],
                               faces[i][3],
                               faces[len(faces)//2+i][3],
                               faces[len(faces)//2+i][2],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][3],
                               faces[i][4],
                               faces[len(faces)//2+i][4],
                               faces[len(faces)//2+i][3],
                              ]))
        
    return side_faces

In [18]:
def make_faces(arr, n_floor_v):
    faces = make_floor_faces(arr)
    faces.extend(make_ceil_faces(arr, n_floor_v))
    faces.extend(make_side_faces(faces))
    
    return faces

In [19]:
def make_face_color(arr_color_flat, arr_mesh, clim):
    arr_color_flat = np.delete(arr_color_flat, np.isnan(arr_mesh.flatten(order='F')))
    
    faces_color = np.concatenate((np.tile(arr_color_flat, 2),
                               np.tile(arr_color_flat, (4,1)).flatten(order='F')))
    
#     arr_color_opacity = (np.abs(arr_color_flat-np.mean([clim[0],clim[1]])) / (np.ptp(clim)/2))*3
    
#     faces_opacity = np.concatenate((np.tile(arr_color_opacity, 2),
#                                np.tile(arr_color_opacity, (4,1)).flatten(order='F')))    
    
#     faces_opacity[faces_opacity>1] = 1.0
    
#     faces_opacity = faces_opacity*0.8
    
    return faces_color

In [20]:
def make_surface(arr_mesh, cube_size, arr_color, clim):
    
    vertices, n_floor_v = make_vertices(arr_mesh, cube_size)
    faces = make_faces(arr_mesh, n_floor_v)
    surf = pv.PolyData(vertices, faces)
    faces_color = make_face_color(arr_color.flatten(order='F'), arr_mesh, clim)
    surf["colors"] = faces_color
    
    return surf

### Event

In [21]:
def get_events(event_csv_path, time_arr, data_shape, lat_min, lat_max, long_min, long_max):
    """ Gets a csv file path of highlighted events that should be annotated in the rendering,
    (as a text bubble on a specified map location) 
    it converts the data to x,y mesh position and timeframe indexing and returns it as a dictionary.

    Args:
        event_csv_path: str path to csv file. Each row in the csv is an event. csv should have 5 columns:
            "first_date" - first date of event in the format e.g. 20001231
            "last_date" - last date of event
            "longitude" - event longitude coordinate
            "latitude" - event latitude coordinate
            "text" - event text
        time_arr: 1D array. Part of the netCDF dataset
        long_arr: 1D array. Part of the netCDF dataset
        lat_arr: 1D array. Part of the netCDF dataset

    Returns:
        event_dict - dict representation of the csv where each key is a timeframe index.
                example: {"0":[[-20,250,150],"event text example"]}
                means that there is only one event, it would be displayed at the first (0) timeframe,
                at location [x,y,z] of the mesh coordinates.

    Note:
        for now also processing timepoints outside of user input timepoints.
    TODO:
        Need to check that the csv format is correct.
        for now not checking that long and lat values are close to values in arrays:
        e.g. [abs(long_arr[idx_long[i]]-val)<max_dist for i,val in enumerate(loc_arr[0])]
    """

    lat_arr = np.linspace(lat_min, lat_max, data_shape[0])
    long_arr = np.linspace(long_min, long_max, data_shape[1])
        
    event_dict = {}

    df = pd.read_csv(event_csv_path, dtype={"first_date": str, "last_date": str})
    for i in df.index:
        try:
            idx_first_date = time_arr.index(df.loc[i, "first_date"])
            idx_last_date = time_arr.index(df.loc[i, "last_date"])
        except Exception as e:
            logging.exception(e)
            logging.warning(f'event csv has an invalid date - row #{i}')
            continue
            
        point = [(np.abs(long_arr - df.loc[i, "longitude"])).argmin(),
                 (np.abs(lat_arr - df.loc[i, "latitude"])).argmin(),
                 150]  # x,y,z
        
        if (not lat_min<point[0]<lat_max) or (not long_min<point[0]<long_max):
            logging.warning(f'event csv has an invalid location - row #{i}')
            continue
                                    
        text = (df.loc[i, "text"]).replace('\\n', '\n')
        for idx in range(idx_first_date, idx_last_date + 1):

            if str(idx) not in event_dict:
                event_dict[str(idx)] = [[point], [text]]
            else:
                event_dict[str(idx)][0].append(point)
                event_dict[str(idx)][1].append(text)

    return event_dict

In [23]:
def remove_actors(p):
    p.remove_actor('mesh')
    p.remove_actor('date')
    
    return p

### Video

In [53]:
# Read datasets:
arr_mesh_ref, lat_arr_mesh, long_arr_mesh, time_arr_mesh = load_nc(path_input_mesh)
arr_color_ref, lat_arr_color, long_arr_color, time_arr_color = load_nc(path_input_color)

lat_min, lat_max = find_min_max(lat_arr_mesh, 
                                lat_arr_color, 
                                crop_coordinates[:2] if crop_coordinates is not None else (np.nan, np.nan))
long_min, long_max = find_min_max(long_arr_mesh, 
                                  long_arr_color, 
                                  crop_coordinates[2:] if crop_coordinates is not None else (np.nan, np.nan))

bg, data_shape = make_BG_mesh(path_input_BG, bg_padding, 
                              lat_min, lat_max, long_min, long_max)

events_per_time_idx_dict = get_events(event_csv_path, time_arr_mesh, data_shape,
                                      lat_min, lat_max, long_min, long_max,
                                     ) if event_csv_path is not None else {}


if is_interp=='temporal':
    arr_color = interpolate_nans_temporal(arr_color_ref)
    
p = create_plotter(frame_rate)
p = add_BG(p, bg)

## missing!! need to verify the range of the mesh and the color arrays.

for i_time in range(5):#(n_timepoints):
    
    arr_mesh = load_arr_2D(arr_mesh_ref, i_time, min_zero=True)
    arr_color = load_arr_2D(arr_color_ref, i_time)
    
    arr_mesh = get_world_part(arr_mesh, (lat_min, lat_max), (long_min, long_max),
                              lat_arr_mesh, long_arr_mesh)
    arr_color = get_world_part(arr_color, (lat_min, lat_max), (long_min, long_max),
                               lat_arr_color, long_arr_color)
    
    arr_mesh = cv2.resize(arr_mesh, (data_shape[1]//cube_size, data_shape[0]//cube_size), 
                                     interpolation=cv2.INTER_NEAREST).T
    arr_color = cv2.resize(arr_color, (data_shape[1]//cube_size, data_shape[0]//cube_size), 
                                        interpolation=cv2.INTER_NEAREST).T
    
    if is_interp=='spatial':
        arr_color = interpolate_nans(arr_color)
    
    surf = make_surface(arr_mesh, cube_size, arr_color, clim)
    
    for iii in range(300//frame_rate):
        if iii==0:
            
            p = remove_actors(p)

            p.add_mesh(surf, scalars="colors", cmap=cmap, clim=clim, show_edges=True, lighting=True, name='mesh')
            if str(i_time) in events_per_time_idx_dict:
                p.add_point_labels(events_per_time_idx_dict[str(i_time)][0],
                                   events_per_time_idx_dict[str(i_time)][1],
                                   name='event_text', italic=True, font_size=40,
                                   shape_color='black', point_color='pink', point_size=40,
                                   shape_opacity=0.5,
                                   render_points_as_spheres=True, always_visible=True, shadow=True)
            if i_time==0:
                p.camera_position = 'xy'
                #p.camera.position = (p.camera_position.position[0],p.camera_position.position[1],p.camera_position.position[2]+2000)
                p.camera.zoom(2)
                p.camera.azimuth -= 30
                p.camera.elevation = -20
        
        p.camera.azimuth += 0.02
        p.write_frame()

pv.close_all()

INFO:root:Read netCDF surface_upward_sensible_heat_flux data. Shape=(460, 601, 1233)
INFO:root:Read netCDF LAI data. Shape=(144, 602, 1293)


True